## Tratamento dos dados

### Importando bibliotecas

In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import adjusted_mutual_info_score
import pandas as pd

### Importando o dataset

In [2]:
df = pd.read_csv('/Users/thomazaraujo/Documents/CIn-UFPE/PIBIC/Fuzzy_Clustering/datasets/cervcancer.csv')
df.head()

,behavior_eating,behavior_personalHygiene,intention_aggregation,intention_commitment,attitude_consistency,attitude_spontaneity,norm_significantPerson,norm_fulfillment,perception_vulnerability,perception_severity,motivation_strength,motivation_willingness,socialSupport_emotionality,socialSupport_appreciation,socialSupport_instrumental,empowerment_knowledge,empowerment_abilities,empowerment_desires,behavior_sexualRisk,ca_cervix
0,13,12,4,7,9,10,1,8,7,3,14,8,5,7,12,12,11,8,10,1
1,11,11,10,14,7,7,5,5,4,2,15,13,7,6,5,5,4,4,10,1
2,15,3,2,14,8,10,1,4,7,2,7,3,3,6,11,3,3,15,10,1
3,11,10,10,15,7,7,1,5,4,2,15,13,7,4,4,4,4,4,10,1
4,11,7,8,10,7,8,1,5,3,2,15,5,3,6,12,5,4,7,8,1


### Retirando variáveis inúteis e transformando classes em números

In [3]:
labels = df["ca_cervix"].values
df.drop("ca_cervix", axis=1, inplace=True)
dados = df.to_numpy()

In [4]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ik}(i=1,...c$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ tal que:
- $u_{ik} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ik} < n$;
- $\sum_{i=1}^cu_{ik} = 1$ para todo $k \in \Omega$.

In [5]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Inicialização dos pesos

O peso é inicializado de maneira fixa $\lambda_i^j$, que é calculado para ser proporcional ao inverso da variância (dispersão) da variável $j$ no cluster $i$, seguindo a única restrição seguinte:
- $\lambda_i^j=1,\forall i, j$

In [6]:
def inicializacao_pesos(num_clusters, num_variaveis):
    pesos = np.ones((num_clusters, num_variaveis)) # inicializa os pesos com 1
    return pesos

### Atualização dos centroides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

### $y_i = \frac{\sum_{k=1}^n(u_{ik})^mx_k}{\sum_{k=1}^n(u_{ik})^m}$

In [7]:
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

### $u_{ik} = \left[\sum_{h=1}^c\left\{\frac{\sum_{j=1}^p\lambda_i^j(x_{jk}-y_{ij})^2}{\sum_{j=1}^p\lambda_h^j(x_{jk}-y_{hj})^2}\right\}^\frac{1}{m-1}\right]^{-1}$

In [8]:
def atualizacao_matriz_pertinencia(dados, centroides, pesos, m):
    diff_sq = (dados[:, np.newaxis, :] - centroides) ** 2 # (xjk - yij)^2
    weighted_diff_sq = pesos * diff_sq # λij
    matriz_distancias = np.sum(weighted_diff_sq, axis=2) # \sum_{j=1}^p
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia / np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

### Atualização dos pesos

Fixo o protótipo e o grau de pertinência, atualize os pesos com base na seguinte equação:

### $\lambda_i^j=\frac{\{\prod_{h=1}^p[\sum_{k=1}^n(u_{ik})^m(x_{hk}-y_{ih})^2]\}^{\frac{1}{p}}}{\sum_{k=1}^n(u_{ik})^m(x_{jk}-y_{ij})^2}$

sob a restrição:

$\prod_{j=1}^p\lambda_i^j=1, \forall i=1,\ldots,c$

In [9]:
def atualizacao_pesos(dados, centroides, matriz_pertinencia, m):
    num_amostras, num_variaveis = dados.shape
    num_clusters = centroides.shape[0]

    # D[i, j] = sum_k( (u_ik)^m * (x_jk - y_ij)^2 )
    
    matriz_pertinencia_m = matriz_pertinencia ** m
    diff_sq = (dados[:, np.newaxis, :] - centroides) ** 2
    weighted_diff_sq = matriz_pertinencia_m[:, :, np.newaxis] * diff_sq

    D = np.sum(weighted_diff_sq, axis=0) # (C, P)
    D = np.fmax(D, np.finfo(np.float64).eps)

    # Numerador_i = { Prod_h( D_ih ) } ^ (1/p)
    # logaritmos para estabilidade numérica (evita underflow/overflow)
    # log(Numerador_i) = (1/p) * sum_h( log(D_ih) )
    
    log_D = np.log(D)
    soma_log_D = np.sum(log_D, axis=1) # soma os logs das dispersões sobre as variáveis 'h' (axis=1)
    numerador = np.exp(soma_log_D / num_variaveis) # média geométrica

    pesos = numerador[:, np.newaxis] / D # peso final
     
    return pesos

### FCM-C²

Ações:
1. Inicialização da matriz de pertinência
2. Inicialização dos pesos
3. Atualização dos centroides
4. Atualização dos pesos
5. Atualização da matriz de pertinência

Critérios de parada:
1. Número máximo de iterações atingido
2. Pouca diferença (erro) entre as matrizes de pertinência de iterações consecutivas

In [10]:
def fcm(dados, num_clusters, m=2, max_iter=10**6, erro=1e-9):
    num_amostras, num_variaveis = dados.shape
    matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters)
    pesos = inicializacao_pesos(num_clusters, num_variaveis)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        pesos = atualizacao_pesos(dados, centroides, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, pesos, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia, pesos

### Índice de Rand Ajustado (IRA)

In [11]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

### Simulação de Monte Carlo

In [12]:
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    ari = []
    ami = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia, pesos = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        idx_rand = indice_rand(labels, predicted_labels)
        ari.append(idx_rand)
        ami_rand = adjusted_mutual_info_score(labels, predicted_labels)
        ami.append(ami_rand)
    mean_ari = np.mean(ari)
    std_ari = np.std(ari)
    mean_ami = np.mean(ami)
    std_ami = np.std(ami)
    return mean_ari, std_ari, mean_ami, std_ami

### Definição de parâmetros e execução do método

In [13]:
num_clusters = 2
num_trials = 100
mean_ari, std_ari, mean_ami, std_ami = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo FCM-C² Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {mean_ari:.4f}")
print(f"Standard Deviation of Rand Index: {std_ari:.4f}")
print("")
print(f"Mean Adjusted Mutual Information: {mean_ami:.4f}")
print(f"Standard Deviation of Adjusted Mutual Information: {std_ami:.4f}")

Monte Carlo FCM-C² Clustering Results (100 trials)
Mean Rand Index: 0.1897
Standard Deviation of Rand Index: 0.0000

Mean Adjusted Mutual Information: 0.1053
Standard Deviation of Adjusted Mutual Information: 0.0000
